# ByteCode Hacks in
![](python-logo.png)

### We will use the following code for our purpose

In [1]:
def myfunc():
    return 6 / 2

print myfunc()

3


- The function just divides two constants and returns the value back to the caller. 
- Now let’s try to print the code object generated for this function. 
- The code object is accessible from `myfunc.__code__`

In [18]:
co = myfunc.__code__
print "co_consts:", co.co_consts
print "co_code:", repr(co.co_code)

co_consts: (None, 10, 2)
co_code: 'd\x01\x00d\x02\x00\x15S'


- The attribute co_consts, contains a tuple of constants used by the function

- The co_code, contains the byte code instructions, generated by compiling the function.

- This just shows that there is a code object associated with every function, that contains the byte code instructions and associated data to execute the function.

# Deciphering the Byte Code

- When represented as a string, the byte code instructions does not make much sense. Let’s try printing it in hex.

In [3]:
for i, ch in enumerate(co.co_code):
    print "    %d: %02X" % (i, ord(ch))

    0: 64
    1: 01
    2: 00
    3: 64
    4: 02
    5: 00
    6: 15
    7: 53


- The first byte code instruction 0x64.
- How do we determine what that actually means?

# Python Dis-assembler Module

In [6]:
import dis
print dis.opname[0x64]

LOAD_CONST


In [7]:
dis.dis(myfunc)

  2           0 LOAD_CONST               1 (6)
              3 LOAD_CONST               2 (2)
              6 BINARY_DIVIDE       
              7 RETURN_VALUE        


# Writing a Dis-assembler

In [21]:
code = co.co_code; state = "opcode"

In [17]:
for op in code:
    op = ord(op)
    if state == "opcode":
        print "%02X" % op, dis.opname[op]
        if op > dis.HAVE_ARGUMENT:
            state = "arg1"
    elif state == "arg1":
        print "%02X" % op
        state = "arg2"
    elif state == "arg2":
        print "%02X" % op
        state = "opcode"

64 LOAD_CONST
01
00
64 LOAD_CONST
02
00
15 BINARY_DIVIDE
53 RETURN_VALUE


# Hacking Code Objects

- Change the tuple (None, 10, 2) instead of (None, 6, 2)

In [19]:
import new

co = myfunc.__code__
myconsts = (None, 10, 2)

In [16]:
co2 = new.code(co.co_argcount,
               co.co_nlocals,
               co.co_stacksize,
               co.co_flags,
               co.co_code,
               myconsts,
               co.co_names,
               co.co_varnames,
               co.co_filename,
               co.co_name,
               co.co_firstlineno,
               co.co_lnotab)

# Injecting the modified code object
myfunc.__code__ = co2

print myfunc()

5
